## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob
from datetime import datetime, timedelta

## Data Read-in

In [2]:
csv_files = glob.glob('*.csv')

df_list = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df_list.append(df)
    
df = pd.concat(df_list,ignore_index=True)

print(len(df))

2806


In [3]:
gdf = gpd.read_file('Miami-Dade_Boundary.geojson')

## Previous month's values

Get values from here:

https://docs.google.com/spreadsheets/d/1GWlqTIzCB29pDVDjd-iBVejVGGsLiAd0P6fq4LM83WI/edit?usp=sharing

In [4]:
print('Input total condo units sold previous (not last) month:')
total_condos_sold_2nd_month = input()

Input total condo units sold previous (not last) month:
598


In [5]:
print('Input total sales volume ($$$ million) from previous (not last) month:')
sales_volume_2nd_month = input()

Input total sales volume ($$$ million) from previous (not last) month:
466093925


In [6]:
print('Input median sales price from previous (not last) month:')
median_sales_price_2nd_month = input()

Input median sales price from previous (not last) month:
400000


In [7]:
print('Input median psf from previous (not last) month:')
median_psf_2nd_month = input()

Input median psf from previous (not last) month:
432


## Data Clean

In [8]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [9]:
df = df.dropna(subset=['SOLD DATE'])

In [10]:
# Get the current date
current_date = datetime.now()

# Calculate the first day of the current month
first_day_of_current_month = current_date.replace(day=1)

# Get the last day of the previous month by subtracting one day from the first day of the current month
last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the month name of the previous month
previous_month_name = last_day_of_previous_month.strftime("%B")

# ---- Get the name of the month, as a string, from two months ago.
# ---- Meaning, if the current month is January, return November.

# Get the first day of the previous month by subtracting one day from the first day of the current month
first_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the first day of the month before the last by subtracting one day from the first day of the previous month
first_day_of_month_before_last = first_day_of_previous_month.replace(day=1) - timedelta(days=1)

# Get the month name of the month before the last
month_before_last_name = first_day_of_month_before_last.strftime("%B")

print(f'Report month: {previous_month_name} ---- Month before report month: {month_before_last_name}')

Report month: February ---- Month before report month: January


In [11]:
# Define list of desired months (excluding current month)
desired_months = [previous_month_name]

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [12]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    741
Name: count, dtype: int64
-------
$/SQUARE FEET
False    739
True       2
Name: count, dtype: int64
-------
YEAR BUILT
False    741
Name: count, dtype: int64
-------


In [13]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Doral/10950-NW-82nd-St-33178/unit-421/home/179455047


In [14]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [15]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [16]:
df_filtered = df_filtered.drop_duplicates().reset_index(drop=True)

In [17]:
# df_filtered = df_filtered[df_filtered['ADDRESS'] != '1901 Brickell Ave Unit B109']

In [17]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
317,PAST SALE,February-26-2024,Condo/Co-op,1470 NE 125 Ter #205,North Miami,FL,33161.0,160.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/1470-NE-...,MARMLS,A11492932,N,Y,25.891296,-80.169137
324,PAST SALE,February-26-2024,Condo/Co-op,20281 E Country Club Dr Unit CAB6,Aventura,FL,33180.0,50000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/20281-E-Cou...,MARMLS,A11491239,N,Y,25.965506,-80.125855
310,PAST SALE,February-23-2024,Condo/Co-op,16900 NE 14th Ave #108,North Miami Beach,FL,33162.0,60000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/16...,Beaches MLS,F10415042,N,Y,25.931418,-80.172957
318,PAST SALE,February-21-2024,Condo/Co-op,251 NE 187th St #807,Miami,FL,33179.0,80000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/251-NE-187th-S...,MARMLS,A11458861,N,Y,25.946879,-80.196152
297,PAST SALE,February-5-2024,Condo/Co-op,1351 NE Miami Gardens Dr Unit 1223E,Miami,FL,33179.0,135000.0,1.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1351-NE-Miami-...,MARMLS,A11491142,N,Y,25.945838,-80.174826
339,PAST SALE,February-16-2024,Condo/Co-op,18746 NE 18th Ave #247,Miami,FL,33179.0,138000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/18746-NE-18th-...,MARMLS,A11445330,N,Y,25.948563,-80.165962
340,PAST SALE,February-13-2024,Condo/Co-op,952 NE 199th St Unit 2J,Miami,FL,33179.0,140000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/952-NE-199th-S...,MARMLS,A11427835,N,Y,25.958380,-80.182929
329,PAST SALE,February-1-2024,Condo/Co-op,2980 Point East Dr Unit D311,Aventura,FL,33160.0,141180.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2980-Point-...,MARMLS,A11463219,N,Y,25.940986,-80.140743
332,PAST SALE,February-15-2024,Condo/Co-op,488 NW 165th Street Rd #116,Miami,FL,33169.0,142000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/488-NW-165th-S...,MARMLS,A11449437,N,Y,25.923303,-80.207214
301,PAST SALE,February-29-2024,Condo/Co-op,8200 SW 210th St #216,Cutler Bay,FL,33189.0,150000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Cutler-Bay/8200-SW-2...,MARMLS,A11509200,N,Y,25.571765,-80.323356


In [20]:
print(df_filtered['URL'].iloc[324])

https://www.redfin.com/FL/Aventura/20281-E-Country-Club-Dr-33180/unit-CAB6/home/42935427


In [21]:
# Correct the prices, if needed
df_filtered.at[317,'PRICE']=(160_000)

# Correct the psf, if needed
df_filtered.at[317,'$/SQUARE FEET']=(160_000/576)

In [21]:
### If needed, drop the row with the lowest price
# min_value_index = df_filtered['PRICE'].idxmin()
# df_filtered = df_filtered.drop(min_value_index)

In [22]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
310,60000.0,16900 NE 14th Ave #108,North Miami Beach,87.0
318,80000.0,251 NE 187th St #807,Miami,94.0
342,165000.0,1690 NE 191st St Unit 413-1,Miami,99.0
335,150000.0,484 NW 165th Street Rd #409,Miami,128.0
334,178000.0,1750 NE 191st St Unit 220-3,Miami,135.0
332,142000.0,488 NW 165th Street Rd #116,Miami,137.0
194,275000.0,1800 NE 114th St #1908,Miami,141.0
293,158000.0,1780 NE 191st St Unit 603-2,Miami,149.0
302,185000.0,952 NE 199th St #411,Miami,155.0
297,135000.0,1351 NE Miami Gardens Dr Unit 1223E,Miami,157.0


In [23]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [23]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [24]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [25]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [26]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [27]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [28]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"{previous_month_name} 2024 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(gdf,tooltip='Miami-Dade County',name='Miami-Dade County').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
# m

In [29]:
m.save('index.html')

## Summary Info

In [30]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [31]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [32]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [33]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Beach/100-Collins-Ave-33139/unit-218/home/43258036


In [34]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
SURF CLUB CONDO, 9001 Collins Ave Unit S-901 Surfside | Price $24,000,000 | $4,058 psf | Year built: 2017
Least Expensive
HAMPTONS WEST CONDO, 20281 E Country Club Dr Unit CAB6 Aventura | Price $50,000 | $278 psf | Year built: 1984
Highest Price Per Square Foot
SURF CLUB CONDO, 9001 Collins Ave Unit S-901 Surfside | Price $24,000,000 | $4,058 psf | Year built: 2017
Lowest Price Per Square Foot
Mar-len Gardens, 16900 NE 14th Ave #108 North Miami Beach | Price $60,000 | $87 psf | Year built: 1965
Newest
Missoni Baia, 700 NE 26 Ter #4801 Miami | Price $2,900,000 | $1,136 psf | Year built: 2023
Oldest
THE MERCURY SOUTH BEACH, 100 Collins Ave #218 Miami Beach | Price $245,000 | $613 psf | Year built: 1921


## Map URL Snagger

In [35]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_month_ending_feb_2024


## Get Summary Data

In [36]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 704
--------
Total sale price: $558,715,499
--------
Median sale price: $402,500
--------
Max sale price: $24,000,000
--------
Min sale price: $50,000
------------------------------------------------
PSF INFO
Max price per square foot: $4,058
--------
Min price per square foot: $87
--------
Median price per square foot: $424
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1921.0
----------
Average building age: 1988.268465909091


In [37]:
most_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']
most_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmax()]['CITY']

least_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']
least_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmin()]['CITY']

# GENERATE STORY TEMPLATE

# STOP! DID YOU UPDATE VALUES FROM THE PREVIOUS MONTH?

In [39]:
## Calculate the increase/descrease between both months
subject_month_sales_volume = df_filtered["PRICE"].sum()
prior_month_sales_volume = int(sales_volume_2nd_month)  # Example value, replace with actual value

subject_month_closings = len(df_filtered)
prior_month_closings = int(total_condos_sold_2nd_month)

# Subtract the smaller value from the larger one
if subject_month_sales_volume > prior_month_sales_volume:
    hed_rf = 'jumps'
    nut_graf = 'rising'
    social = 'higher'
    seo = 'increases'
elif prior_month_sales_volume > subject_month_sales_volume:
    hed_rf = 'drops'
    nut_graf = 'falling'
    social = 'lower'
    seo = 'decreases'
else:
    hed_rf = 'equals'
    nut_graf = 'equaling'
    social = 'equal'
    seo = "doesn't change"
    
    
if subject_month_closings > prior_month_closings:
    dek_rf = 'rises'
elif prior_month_closings > subject_month_closings:
    dek_rf ='falls'
else:
    dek_rf = 'equals'

In [40]:
prior_month_sales_volume

466093925

In [41]:
prior_month_closings

598

In [42]:
story_string = f'''
\033[1mHED:\033[0m Miami-Dade County condo sales dollar volume {hed_rf} in {previous_month_name} to ${round(df_filtered["PRICE"].sum()/1_000_000)}M 
\033[1mDEK:\033[0m Number of closings {dek_rf} to {len(df_filtered)} from {total_condos_sold_2nd_month} in {month_before_last_name}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County {previous_month_name} Condo Sales Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales volume {hed_rf} to ${round(df_filtered["PRICE"].sum()/1_000_000)} million in {previous_month_name}.
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m Miami-Dade County had ${round(df_filtered["PRICE"].sum()/1_000_000):,.0f} million in condo sales in {previous_month_name}, {social} than ${round(int(sales_volume_2nd_month)/1_000_000)} million from {month_before_last_name}, @afarence reports 
\033[1mART:\033[0m 

CHANGE ME

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, monthly condo sales, condos, condo market, {most_expensive_muni_condo_name.title()}, {most_expensive_muni_location}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m


Miami-Dade County’s NEWS PEG HERE.

{previous_month_name} condo sales totaled ${round(df_filtered["PRICE"].sum()/1_000_000)} million, {nut_graf} from ${int(sales_volume_2nd_month)/1_000_000:.0f} million in {month_before_last_name}. Brokers closed {len(df_filtered)} sales last month, down from {total_condos_sold_2nd_month} sales in {month_before_last_name} and XXX in XXX, Multiple Listing Service data from Redfin show.

{previous_month_name} sale prices ranged from ${df_filtered["PRICE"].min():,.2f} to ${df_filtered["PRICE"].max()/1_000_000:,.2f} million, with a median sale price of ${df_filtered["PRICE"].median():,.0f}. The price per square foot ranged from ${df_filtered["$/SQUARE FEET"].min():,.0f} to ${df_filtered["$/SQUARE FEET"].max():,.0f}, with a median of ${df_filtered["$/SQUARE FEET"].median():,.0f} per square foot.

In {month_before_last_name}, sales closed with a median price of ${int(median_sales_price_2nd_month):,.0f}, and ${median_psf_2nd_month} per square foot.

A ${df_filtered["PRICE"].max()/1_000_000:,.2f} million closing at {most_expensive_muni_condo_name.title()} took the title of priciest sale last month. {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} sold for ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} per square foot after XX days on the market. XXX with XXX had the listing, and XXX with XXX represented the buyer.

The sale price was \033[1mmore/less\033[0m than {month_before_last_name}'s priciest sale. WHEN XXXX 

In contrast, {previous_month_name}'s cheapest sale was an {least_expensive_muni_condo_name} in {least_expensive_muni_location}, at {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']}. Unit XXX traded for ${df_filtered['PRICE'].min():,.0f} — or ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} per square foot — after XXX days on the market. XXX with XXX handled both sides of the deal.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="800" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of {previous_month_name}’s notable condo sales:
'''

In [43]:
print(story_string)

print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")


HED: Miami-Dade County condo sales dollar volume jumps in February to $559M 
DEK: Number of closings rises to 704 from 598 in January
FEATURED HED:
SEO HED: Miami-Dade County February Condo Sales Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales volume jumps to $559 million in February.
AUTHOR: Adam Farence
RESEARCH: 
Social: Miami-Dade County had $559 million in condo sales in February, higher than $466 million from January, @afarence reports 
ART: 

CHANGE ME

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, monthly condo sales, condos, condo market, Surf Club Condo, Surfside

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:


Miami-Dade County’s NEWS PEG HERE.

February condo sales totaled $559 million, rising from $466 million in January. Brokers closed 704 sales last month, down from 598 sales in January and XXX in XXX, Multiple Listi

# Print links for notable sales

## Top sale

In [ ]:
print(df_filtered.loc[df_filtered['PRICE'].idxmax()]['URL'])

## Cheapest sale

In [ ]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

## Highest PSF

In [ ]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['URL'])

## Lowest PSF

In [ ]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['URL'])

## Newest

In [ ]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])

## Oldest

In [ ]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

## Time on Market Calculator

In [ ]:
from datetime import datetime, timedelta

################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 11, 4) ## List (Earlier) date
date2 = datetime(2024, 1, 9) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)